-todo-
clean the df some
fix dtypes
remove obsolete stuff


In [18]:
%run currency_scrape.ipynb

<Response [200]>


In [19]:
import requests
#from bs4 import BeautifulSoup

In [20]:
import pandas as pd
import json
import numpy as np

In [21]:
gameName='Hitman 3'
gameNameAlt='Hitman 3'

-------- Start Steam ------

In [22]:
steamAppsResponse = requests.get(
    "https://api.steampowered.com/ISteamApps/GetAppList/v2/")

steamApps = steamAppsResponse.json()
steamApps = steamApps['applist']['apps']  # ['app']
#steamApps

In [23]:
steam_df=pd.json_normalize(steamApps)
#steam_df.set_index('appid',inplace=True)

steam_df.tail()

,appid,name
154471,1351000,Gangs of Sherwood
154472,1839950,Terminator: Dark Fate - Defiance
154473,1599660,Sackboy™: A Big Adventure
154474,1981100,Rustforge Playtest
154475,1926090,Ultracore


import pickle

picklepath = "pickled/pickledf.pkl";

steam_df.to_pickle(picklepath)

other_df=pd.read_pickle(picklepath)
other_df

------remove all empty names rows from game list-----

In [26]:
steam_df.replace('', np.nan, inplace=True)
steam_df.dropna(subset=['name'], inplace=True)
steam_df.head()

,appid,name
6,216938,Pieterw test app76 ( 216938 )
7,660010,test2
8,660130,test3
26,895070,Sharknado
27,895110,Magical Battle Cry


-------- filter for specific game ---------

In [27]:
#gameName='Syberia: The World Before'

In [28]:
#exact match
steamGamedf=steam_df.loc[steam_df['name'].str.lower().eq(gameName.lower())]
if steamGamedf.empty:
    steamGamedf=steam_df.loc[steam_df['name'].str.lower().eq(gameNameAlt.lower())]
if steamGamedf.empty:
    #contains match
    steamGamedf=steam_df.loc[steam_df['name'].str.contains(gameName, case=False)]
if steamGamedf.empty:
    #contains match
    steamGamedf=steam_df.loc[steam_df['name'].str.contains(gameNameAlt, case=False)]
steamGamedf

,appid,name
69392,1659040,HITMAN 3
110586,1659040,HITMAN 3


---- make copy with url inserted------

In [29]:
steamGamedf['url'] = steamGamedf.agg('https://store.steampowered.com/api/appdetails?appids={0[appid]}'.format, axis=1)
steamGamedf

C:\Users\robbo\AppData\Local\Temp\ipykernel_20496\3546206248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steamGamedf['url'] = steamGamedf.agg('https://store.steampowered.com/api/appdetails?appids={0[appid]}'.format, axis=1)


,appid,name,url
69392,1659040,HITMAN 3,https://store.steampowered.com/api/appdetails?...
110586,1659040,HITMAN 3,https://store.steampowered.com/api/appdetails?...


In [30]:
#steamGamedf['url'].values[0]

In [31]:
frames=[]
for index, row in steamGamedf.iterrows():
    #print(row["name"], row["url"])
    steamResponse = requests.get(row['url']).json() 
    if steamResponse[str(row["appid"])]['success']:
        datac=steamResponse[str(row["appid"])]['data']
        #if "price_overview" in datac:
        steamSdf=pd.json_normalize(datac)
        steamSdf.keys()
        frames.append(steamSdf)
        
single = pd.concat(frames, ignore_index=True)
single

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,metacritic.url,recommendations.total,achievements.total,achievements.highlighted,release_date.coming_soon,release_date.date,support_info.url,support_info.email,content_descriptors.ids,content_descriptors.notes
0,game,HITMAN 3,1659040,18,False,full,"[1829591, 1829592, 1829590, 1843460, 1829595, ...","<h1>Standard Edition</h1><p><img src=""https://...",<u><strong>WORLD OF ASSASSINATION</strong></u>...,"Death Awaits. Agent 47 returns in HITMAN 3, th...",...,https://www.metacritic.com/game/pc/hitman-3?ft...,5274,83,"[{'name': 'The Result of Previous Training', '...",False,"20 Jan, 2022",https://www.ioi.dk/player-support/,,"[2, 5]",This Game may contain content not appropriate ...
1,game,HITMAN 3,1659040,18,False,full,"[1829591, 1829592, 1829590, 1843460, 1829595, ...","<h1>Standard Edition</h1><p><img src=""https://...",<u><strong>WORLD OF ASSASSINATION</strong></u>...,"Death Awaits. Agent 47 returns in HITMAN 3, th...",...,https://www.metacritic.com/game/pc/hitman-3?ft...,5274,83,"[{'name': 'The Result of Previous Training', '...",False,"20 Jan, 2022",https://www.ioi.dk/player-support/,,"[2, 5]",This Game may contain content not appropriate ...


In [32]:
single.keys()

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'controller_support', 'dlc', 'detailed_description', 'about_the_game',
       'short_description', 'supported_languages', 'header_image', 'website',
       'legal_notice', 'developers', 'publishers', 'demos', 'packages',
       'package_groups', 'categories', 'genres', 'screenshots', 'movies',
       'background', 'background_raw', 'pc_requirements.minimum',
       'pc_requirements.recommended', 'mac_requirements.minimum',
       'mac_requirements.recommended', 'linux_requirements.minimum',
       'linux_requirements.recommended', 'price_overview.currency',
       'price_overview.initial', 'price_overview.final',
       'price_overview.discount_percent', 'price_overview.initial_formatted',
       'price_overview.final_formatted', 'platforms.windows', 'platforms.mac',
       'platforms.linux', 'metacritic.score', 'metacritic.url',
       'recommendations.total', 'achievements.total',
       'achievements.highlighted

In [33]:
if 'price_overview.final' in single.columns:
    steamSE = single.assign(priceSEK=single["price_overview.final"]/100*rate_sek)
    #steamSE.round({'priceSEK': 1})
    steamSE.priceSEK=steamSE.priceSEK.round()
else:
    print('No price')
steamSE

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,recommendations.total,achievements.total,achievements.highlighted,release_date.coming_soon,release_date.date,support_info.url,support_info.email,content_descriptors.ids,content_descriptors.notes,priceSEK
0,game,HITMAN 3,1659040,18,False,full,"[1829591, 1829592, 1829590, 1843460, 1829595, ...","<h1>Standard Edition</h1><p><img src=""https://...",<u><strong>WORLD OF ASSASSINATION</strong></u>...,"Death Awaits. Agent 47 returns in HITMAN 3, th...",...,5274,83,"[{'name': 'The Result of Previous Training', '...",False,"20 Jan, 2022",https://www.ioi.dk/player-support/,,"[2, 5]",This Game may contain content not appropriate ...,263.0
1,game,HITMAN 3,1659040,18,False,full,"[1829591, 1829592, 1829590, 1843460, 1829595, ...","<h1>Standard Edition</h1><p><img src=""https://...",<u><strong>WORLD OF ASSASSINATION</strong></u>...,"Death Awaits. Agent 47 returns in HITMAN 3, th...",...,5274,83,"[{'name': 'The Result of Previous Training', '...",False,"20 Jan, 2022",https://www.ioi.dk/player-support/,,"[2, 5]",This Game may contain content not appropriate ...,263.0


In [34]:
steamSE.filter(['name','type','controller_support',
       'price_overview.final',
       'price_overview.discount_percent',
       'price_overview.final_formatted','priceSEK'],axis=1)

,name,type,controller_support,price_overview.final,price_overview.discount_percent,price_overview.final_formatted,priceSEK
0,HITMAN 3,game,full,2399,60,"23,99€",263.0
1,HITMAN 3,game,full,2399,60,"23,99€",263.0


In [35]:
steamSE['priceSEK'].values[0]

263.0